# Lambda School Data Science - Making Data-backed Assertions

This is, for many, the main point of data science - to create and support reasoned arguments based on evidence. It's not a topic to master in a day, but it is worth some focused time thinking about and structuring your approach to it.

## Lecture - generating a confounding variable

The prewatch material told a story about a hypothetical health condition where both the drug usage and overall health outcome were related to gender - thus making gender a confounding variable, obfuscating the possible relationship between the drug and the outcome.

Let's use Python to generate data that actually behaves in this fashion!

In [2]:
import random
dir(random)  # Reminding ourselves what we can do here

['BPF',
 'LOG4',
 'NV_MAGICCONST',
 'RECIP_BPF',
 'Random',
 'SG_MAGICCONST',
 'SystemRandom',
 'TWOPI',
 '_BuiltinMethodType',
 '_MethodType',
 '_Sequence',
 '_Set',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_acos',
 '_bisect',
 '_ceil',
 '_cos',
 '_e',
 '_exp',
 '_inst',
 '_itertools',
 '_log',
 '_pi',
 '_random',
 '_sha512',
 '_sin',
 '_sqrt',
 '_test',
 '_test_generator',
 '_urandom',
 '_warn',
 'betavariate',
 'choice',
 'choices',
 'expovariate',
 'gammavariate',
 'gauss',
 'getrandbits',
 'getstate',
 'lognormvariate',
 'normalvariate',
 'paretovariate',
 'randint',
 'random',
 'randrange',
 'sample',
 'seed',
 'setstate',
 'shuffle',
 'triangular',
 'uniform',
 'vonmisesvariate',
 'weibullvariate']

In [3]:
# Let's think of another scenario:
# We work for a company that sells accessories for mobile phones.
# They have an ecommerce site, and we are supposed to analyze logs
# to determine what sort of usage is related to purchases, and thus guide
# website development to encourage higher conversion.

# The hypothesis - users who spend longer on the site tend
# to spend more. Seems reasonable, no?

# But there's a confounding variable! If they're on a phone, they:
# a) Spend less time on the site, but
# b) Are more likely to be interested in the actual products!

# Let's use namedtuple to represent our data

from collections import namedtuple
# purchased and mobile are bools, time_on_site in seconds
User = namedtuple('User', ['purchased','time_on_site', 'mobile'])

example_user = User(False, 12, False)
print(example_user)

User(purchased=False, time_on_site=12, mobile=False)


In [4]:
# And now let's generate 1000 example users
# 750 mobile, 250 not (i.e. desktop)
# A desktop user has a base conversion likelihood of 10%
# And it goes up by 1% for each 15 seconds they spend on the site
# And they spend anywhere from 10 seconds to 10 minutes on the site (uniform)
# Mobile users spend on average half as much time on the site as desktop
# But have three times as much base likelihood of buying something

users = []

for _ in range(250):
  # Desktop users
  time_on_site = random.uniform(10, 600)
  purchased = random.random() < 0.1 + (time_on_site / 1500)
  users.append(User(purchased, time_on_site, False))
  
for _ in range(750):
  # Mobile users
  time_on_site = random.uniform(5, 300)
  purchased = random.random() < 0.3 + (time_on_site / 1500)
  users.append(User(purchased, time_on_site, True))
  
random.shuffle(users)
print(users[:10])

[User(purchased=False, time_on_site=219.26195595830453, mobile=True), User(purchased=False, time_on_site=135.29641193209144, mobile=True), User(purchased=True, time_on_site=110.47777351938323, mobile=True), User(purchased=True, time_on_site=108.19576256005742, mobile=True), User(purchased=True, time_on_site=235.8723046681534, mobile=True), User(purchased=True, time_on_site=73.38702259630175, mobile=True), User(purchased=True, time_on_site=78.27715384704798, mobile=True), User(purchased=True, time_on_site=256.42543684207027, mobile=True), User(purchased=False, time_on_site=137.03835894770188, mobile=True), User(purchased=False, time_on_site=206.49157503121694, mobile=True)]


In [5]:
# Let's put this in a dataframe so we can look at it more easily
import pandas as pd
user_data = pd.DataFrame(users)
user_data.head()

,purchased,time_on_site,mobile
0,False,219.261956,True
1,False,135.296412,True
2,True,110.477774,True
3,True,108.195763,True
4,True,235.872305,True


In [6]:
# Let's use crosstabulation to try to see what's going on
pd.crosstab(user_data['purchased'], user_data['time_on_site'])

time_on_site,5.849917597613611,7.361144926761185,8.135604138444243,8.28307380874351,8.545415431219563,8.597618217729565,8.975190025434,8.978830536380878,9.230429588032306,9.236119381631276,...,584.6156071010173,587.4765815750768,589.1632505614397,591.1448206263542,591.3849939380938,594.2864022142728,595.0960123487864,595.6138971460057,598.4462134091312,598.8973547275862
purchased,,,,,,,,,,,,,,,,,,,,,
False,1,0,1,0,1,1,1,1,1,1,...,0,0,0,1,0,0,1,0,1,0
True,0,1,0,1,0,0,0,0,0,0,...,1,1,1,0,1,1,0,1,0,1


In [7]:
# OK, that's not quite what we want
# Time is continuous! We need to put it in discrete buckets
# Pandas calls these bins, and pandas.cut helps make them

time_bins = pd.cut(user_data['time_on_site'], 5)  # 5 equal-sized bins
pd.crosstab(user_data['purchased'], time_bins)

time_on_site,"(5.257, 124.459]","(124.459, 243.069]","(243.069, 361.678]","(361.678, 480.288]","(480.288, 598.897]"
purchased,,,,,
False,228,232,102,28,25
True,102,147,85,21,30


In [8]:
# We can make this a bit clearer by normalizing (getting %)
pd.crosstab(user_data['purchased'], time_bins, normalize='columns')

time_on_site,"(5.257, 124.459]","(124.459, 243.069]","(243.069, 361.678]","(361.678, 480.288]","(480.288, 598.897]"
purchased,,,,,
False,0.690909,0.612137,0.545455,0.571429,0.454545
True,0.309091,0.387863,0.454545,0.428571,0.545455


In [9]:
# That seems counter to our hypothesis
# More time on the site can actually have fewer purchases

# But we know why, since we generated the data!
# Let's look at mobile and purchased
pd.crosstab(user_data['purchased'], user_data['mobile'], normalize='columns')

mobile,False,True
purchased,,
False,0.668,0.597333
True,0.332,0.402667


In [10]:
# Yep, mobile users are more likely to buy things
# But we're still not seeing the *whole* story until we look at all 3 at once

# Live/stretch goal - how can we do that?

pd.crosstab(user_data['mobile'], [user_data['purchased'],
                                 time_bins], rownames=['device'],
                                 colnames=['purchased', 'time_on_site'])

purchased               False                                        \
time_on_site (5.257, 124.459] (124.459, 243.069] (243.069, 361.678]   
device                                                                
False                      40                 37                 37   
True                      188                195                 65   

purchased                                                     True   \
time_on_site (361.678, 480.288] (480.288, 598.897] (5.257, 124.459]   
device                                                                
False                        28                 25                6   
True                          0                  0               96   

purchased                                                              \
time_on_site (124.459, 243.069] (243.069, 361.678] (361.678, 480.288]   
device                                                                  
False                        11                 15                 21   
True                        136                 70                  0   

purchased                        
time_on_site (480.288, 598.897]  
device                           
False                        30  
True                          0

## Assignment - what's going on here?

Consider the data in `persons.csv` (already prepared for you, in the repo for the week). It has four columns - a unique id, followed by age (in years), weight (in lbs), and exercise time (in minutes/week) of 1200 (hypothetical) people.

Try to figure out which variables are possibly related to each other, and which may be confounding relationships.

In [0]:
# TODO - your code here
# Use what we did live in lecture as an example

# HINT - you can find the raw URL on GitHub and potentially use that
# to load the data with read_csv, or you can upload it yourself

### Assignment questions

After you've worked on some code, answer the following questions in this text block:

1.  What are the variable types in the data?
2.  What are the relationships between the variables?
3.  Which relationships are "real", and which spurious?


## Stretch goals and resources

Following are *optional* things for you to take a look at. Focus on the above assignment first, and make sure to commit and push your changes to GitHub.

- [Spurious Correlations](http://tylervigen.com/spurious-correlations)
- [NIH on controlling for confounding variables](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4017459/)

Stretch goals:

- Produce your own plot inspired by the Spurious Correlation visualizations (and consider writing a blog post about it - both the content and how you made it)
- Pick one of the techniques that NIH highlights for confounding variables - we'll be going into many of them later, but see if you can find which Python modules may help (hint - check scikit-learn)